In [1]:

!pip3 install notebook numpy pandas matplotlib scikit-learn lightgbm --upgrade  

!pip3 install numpy pandas sklearn lightgbm pyarrow
!/Library/Frameworks/Python.framework/Versions/3.11/bin/python3.11 -m pip install pyarrow

import sys; sys.executable

import pandas as pd
import numpy as np



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:


import numpy as np

from split_data import split_data

from model import modeling

from training import prepare_train_test

import lightgbm as lgb


data = pd.read_feather('house_sales.ftr')
data.shape


columns = ['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Tax assessed value', 'Total interior livable area', 'Elementary School Score'
          , 'Lot size', 'Last Sold Price', 'Zip']


df0 = data[columns].copy()
# uncomment the below line to save memory
# del data




df0 = df0.copy()



## Sold Price

c = 'Sold Price'
    
df0[c] = np.log10(df0[c].replace(
        r'[$,-]', '', regex=True).replace(
        r'^\s*$', np.nan, regex=True).astype(float))

df1 = df0[(df0['Sold Price'] >= 4 ) & (df0['Sold Price'] <= 8 )] # use the prices between 10^4 and 10^8 only



# Replace spaces in column names with underscores
#df1.columns = df1.columns.str.replace(' ', '_')


## Type ##
df1.loc[df1['Type'] == 'Single Family', 'Type'] = 'SingleFamily'
# df1['Type'] = df1['Type'].replace('Multi Family', 'MultiFamily').astype(str)

df1 = df1[df1['Type'].isin(['SingleFamily', 'Condo', 'MultiFamily', 'Townhouse'])]

# convert to categorical and then conver to integers
df1['Type'] = df1['Type'].astype('category').cat.codes


## Year Built ##
# Convert 'No Data' in 'Year built' to NaN for easy processing
df1['Year built'] = df1['Year built'].replace('No Data', np.nan)

# Calculate the mean of 'Year built' excluding NaNs
mean_year_built = df1['Year built'].astype(float).mean()

# Replace NaNs with the mean
df1['Year built'].fillna(round(mean_year_built), inplace=True)

# Convert 'Year built' back to int type
df1['Year built'] = df1['Year built'].astype('int')

# Filter out the irregular values
df1 = df1[(df1['Year built'] > 1900) & (df1['Year built'] < 2023)]

## Bedrooms ##
# we see a lot of noises; let's keep only the reasonable ones[0, 10] bedrooms
df1  = df1[df1['Bedrooms'].isin([str(i) for i in range(7)])]
df1['Bedrooms'] = df1['Bedrooms'].astype('int')



## Tax Assessed Value ##
# Convert the column to string type
df1['Tax assessed value'] = df1['Tax assessed value'].astype(str)

df1 = df1[df1['Tax assessed value'] != 'None']

# Remove non-numeric characters and convert to int
df1['Tax assessed value'] = df1['Tax assessed value'].str.replace('$', '').str.replace(',', '').astype(int)

# Filter the dataframe based on your conditions
df1 = df1[(df1['Tax assessed value'] > 100000) & (df1['Year built'] < 300000000)]




## Total interior livable area ##
# Convert the column to string type
df1['Total interior livable area'] = df1['Total interior livable area'].astype(str)

# Replace "None" with NaN
df1['Total interior livable area'] = df1['Total interior livable area'].replace('None', np.nan)

# Remove SQFT
df1['Total interior livable area'] = df1['Total interior livable area'].str.replace('sqft', '').str.replace(',', '')

# Remove NA
df1 = df1.dropna(subset=['Total interior livable area'])

# Convert to integer
df1['Total interior livable area'] = df1['Total interior livable area'].astype(int)



## Elementary School Score ##
# Don't need to manipulate Elementary School Score
df1['Elementary School Score'] = df1['Elementary School Score'].astype(float)



## Lot Size ##
# Convert the 'Lot size' column to string type
df1['Lot size'] = df1['Lot size'].astype(str)

def convert_lot_size(value):
    if value is None or 'None' in value:
        return None
    elif 'Acre' in value:
        # Handle values like '3,080'
        num_value = value.split(' ')[0].replace(',', '')
        return float(num_value) * 43560
    else:
        return float(value.replace('sqft', '').replace(',', ''))

df1['Lot size'] = df1['Lot size'].apply(convert_lot_size)

# Convert to integer, but first drop NaN values
df1 = df1.dropna(subset=['Lot size'])
df1['Lot size'] = df1['Lot size'].astype(int)




## Last Sold Price ##
# Ensure the column is of type str
df1['Last Sold Price'] = df1['Last Sold Price'].astype(str)

# Filter out rows where the 'Last Sold Price' column is 'nan' or 'None'
df1 = df1[df1['Last Sold Price'] != 'nan']
df1 = df1[df1['Last Sold Price'] != 'None']
df1 = df1[df1['Last Sold Price'] != '--']

# Remove non-numeric characters and convert to int
df1['Last Sold Price'] = df1['Last Sold Price'].str.replace('$', '').str.replace(',', '').astype(int)




### Zip ###
df1['Zip'] = df1['Zip'].astype('category').cat.codes

## Train // Test

train, test = split_data(df1, pd.Timestamp(2020, 6, 1))

# need to specify the selected features
features = ["Type", "Year built", 'Bedrooms', 'Last Sold Price', 'Tax assessed value','Elementary School Score','Lot size', 'Total interior livable area', 'Zip' ] 
X_train, y_train, X_test, y_test = prepare_train_test(train, test, features)
# run modeling
predictor = modeling(X_train, y_train, X_test, y_test)
X_train.shape


/Users/samkempton/dataMining/split_data.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')


Starting training...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000634 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1420
[LightGBM] [Info] Number of data points in the train set: 29168, number of used features: 9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 5.864959
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.011694
Starting predicting...
The RMSE of prediction is: 0.10813894005141987


(29168, 9)

In [3]:
''' Summary 

 In my endeavor to craft an optimal model, I utilized a robust set of features, including "Type", "Year built",
"Bedrooms", "Tax assessed value", "Total interior livable area", "Elementary School Score", "Lot size", "Last Sold Price",
and a categorized "Zip" code. During the data preparation process, I put in significant effort to ensure the dataset's integrity.
I meticulously cleared non-numeric characters from the "Sold Price" and restricted its range to values between 10^4 and 10^8. 
I also addressed inconsistencies in the "Type" column, standardizing variations such as "Single Family" and "Multi Family".
For the "Year built" attribute, I enhanced its accuracy by replacing "No Data" values with the average year from the dataset.
With the "Bedrooms" column, I kept it realistic by limiting entries to a range of 0 to 6 bedrooms. Columns like "Tax assessed value",
"Total interior livable area", and "Last Sold Price" were refined by stripping non-numeric values and adapting them to appropriate data 
types. I took the extra step of converting "Lot size" values mentioned in acres to square feet to maintain uniformity. Additionally, the
transformation of "Zip" codes into categorical data greatly benefited the model's adaptability. For the training phase, I harnessed data 
starting from June 2020, yielding a training dataset size of 29,168. Impressively, my model managed to achieve an RMSE of approximately
0.10, showcasing a notable improvement. From the insights drawn, it became clear that "Last Sold Price" and "Tax assessed value" were 
pivotal features shaping the model's predictive capacity.

'''

' Summary \n\n In my endeavor to craft an optimal model, I utilized a robust set of features, including "Type", "Year built",\n"Bedrooms", "Tax assessed value", "Total interior livable area", "Elementary School Score", "Lot size", "Last Sold Price",\nand a categorized "Zip" code. During the data preparation process, I put in significant effort to ensure the dataset\'s integrity.\nI meticulously cleared non-numeric characters from the "Sold Price" and restricted its range to values between 10^4 and 10^8. \nI also addressed inconsistencies in the "Type" column, standardizing variations such as "Single Family" and "Multi Family".\nFor the "Year built" attribute, I enhanced its accuracy by replacing "No Data" values with the average year from the dataset.\nWith the "Bedrooms" column, I kept it realistic by limiting entries to a range of 0 to 6 bedrooms. Columns like "Tax assessed value",\n"Total interior livable area", and "Last Sold Price" were refined by stripping non-numeric values and 